# Ego net creator

COSC2671 Social Media and Network Analytics

Lukas Krodinger, 3931415

In [307]:
import math
import networkx as nx
import json

In [308]:
# Assigns a logarithmic value depending on the follower count
def calculate_importance(follower_count):
    if follower_count <= 0:
        follower_count = 1
    return math.ceil(math.log10(follower_count))

def load(filename):
    datas = []
    with open(filename, 'r') as f:
        for line in f:
            data = json.loads(line)
            datas.append(data)
    return datas

In [309]:
class GraphCreator:
    graph = None
    seen_nodes = set()
    seen_edges = set()

    def __init__(self, graph):
        self.graph = graph

    def add_node(self, node, **attr):
        user_id = int(user.get('id'))
        if user_id not in self.seen_nodes:
            self.graph.add_node(node, **attr)
            self.seen_nodes.add(user_id)

    def add_edge(self, edge_from, edge_to):
        edge = (int(edge_from), int(edge_to))
        if edge not in self.seen_edges:
            self.graph.add_edge(edge_from, edge_to)
            self.seen_edges.add(edge)

    def add_user(self, user):
        user_name = user.get('username')
        user_id = int(user.get('id'))
        followers_count = user.get('public_metrics').get('followers_count')
        following_count = user.get('public_metrics').get('following_count')
        tweet_count = user.get('public_metrics').get('tweet_count')
        self.add_node(user_id, username=user_name, followers_count=followers_count, following_count=following_count, tweet_count=tweet_count, color_id=calculate_importance(followers_count))

    def add_follower(self, ego, user):
        ego_id = int(ego.get('id'))
        user_id = int(user.get('id'))
        self.add_user(user)
        self.add_edge(ego_id, user_id)

    def add_following(self, ego, user):
        ego_id = int(ego.get('id'))
        user_id = int(user.get('id'))
        self.add_user(user)
        self.add_edge(user_id, ego_id)

Parameters:

In [310]:
# Username of the ego
username_or_id = "CricketAus"

# Where to store the data
input_filename = "most_common_users_filtered.json"
directory = "../graph_data"
followers_prefix = "followers"
following_prefix = 'following'

graph_filename= 'ego.graphml'

In [311]:
# Create graph
graph = nx.DiGraph()
graph_creator = GraphCreator(graph)

seen_nodes = set()
seen_edges = set()

In [312]:
egos = load(input_filename)

In [313]:
for ego in egos:
    ego_username = ego.get("username")

    graph_creator.add_user(ego)

    followers_path = directory + "/" + followers_prefix + "_" + ego_username + ".json"
    following_path = directory + "/" + following_prefix + "_" + ego_username + ".json"

    followers = load(followers_path)
    following = load(following_path)

    for user in followers:
        graph_creator.add_follower(ego, user)

    for user in following:
        graph_creator.add_following(ego, user)

In [314]:
# Remove all nodes with less than two edges
remove = [node for node,degree in dict(graph.degree()).items() if degree < 2]
graph.remove_nodes_from(remove)

In [315]:
# Store the graph
with open(graph_filename, 'wb') as fOut:
    nx.write_graphml(graph, fOut)

print("Done!")

Done!
